In [128]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import mean_squared_error
color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')

In [129]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/My Drive/Crime_Data_from_2010_to_2019.csv')
# df = pd.read_csv('cleaned_crime_data.csv')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [130]:
newdf=df[['DATE OCC','LAT','LON']]

newdf['DATE OCC']=pd.to_datetime(newdf['DATE OCC'])



<ipython-input-130-84eb16e627dc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf['DATE OCC']=pd.to_datetime(newdf['DATE OCC'])


In [131]:
def create_features(df):
    """
    Create time series features based on time series index.
    """
    df = df.copy()
    df['dayofweek'] = df['DATE OCC'].dt.dayofweek
    df['quarter'] = df['DATE OCC'].dt.quarter
    df['month'] = df['DATE OCC'].dt.month
    df['year'] = df['DATE OCC'].dt.year
    df['dayofyear'] = df['DATE OCC'].dt.dayofyear
    df['dayofmonth'] = df['DATE OCC'].dt.day
    df['weekofyear'] = df['DATE OCC'].dt.isocalendar().week
    return df

df = create_features(newdf)
df

,DATE OCC,LAT,LON,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear
0,2010-02-20,33.9825,-118.2695,5,1,2,2010,51,20,7
1,2010-09-12,33.9599,-118.3962,6,3,9,2010,255,12,36
2,2010-08-09,34.0224,-118.2524,0,3,8,2010,221,9,32
3,2010-01-05,34.1016,-118.3295,1,1,1,2010,5,5,1
4,2010-01-02,34.0387,-118.2488,5,1,1,2010,2,2,53
...,...,...,...,...,...,...,...,...,...,...
2117584,2019-03-28,34.0962,-118.3490,3,1,3,2019,87,28,13
2117585,2019-08-14,34.0871,-118.3732,2,3,8,2019,226,14,33
2117586,2019-01-06,34.0637,-118.2870,6,1,1,2019,6,6,1
2117587,2019-10-16,34.2266,-118.5085,2,4,10,2019,289,16,42


In [132]:
train = df.loc[df.index < 1694071]
test = df.loc[df.index >= 1694071]

# fig, ax = plt.subplots(figsize=(15, 5))
# train.plot(ax=ax, label='Training Set', title='Data Train/Test Split')
# test.plot(ax=ax, label='Test Set')
# ax.axvline(1800000, color='black', ls='--')
# ax.legend(['Training Set', 'Test Set'])
# plt.show()
train

,DATE OCC,LAT,LON,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear
0,2010-02-20,33.9825,-118.2695,5,1,2,2010,51,20,7
1,2010-09-12,33.9599,-118.3962,6,3,9,2010,255,12,36
2,2010-08-09,34.0224,-118.2524,0,3,8,2010,221,9,32
3,2010-01-05,34.1016,-118.3295,1,1,1,2010,5,5,1
4,2010-01-02,34.0387,-118.2488,5,1,1,2010,2,2,53
...,...,...,...,...,...,...,...,...,...,...
1694066,2018-05-12,34.0554,-118.2789,5,2,5,2018,132,12,19
1694067,2018-07-15,34.0520,-118.1958,6,3,7,2018,196,15,28
1694068,2018-08-07,34.1984,-118.5579,1,3,8,2018,219,7,32
1694069,2018-06-05,34.0573,-118.1935,1,2,6,2018,156,5,23


In [133]:
# train = create_features(train)

# test = create_features(test)



FEATURES = ['dayofyear', 'dayofweek', 'quarter', 'month', 'year']
TARGETS = ['LAT','LON']

X_train = train[FEATURES]
y_train = train[TARGETS]

X_test = test[FEATURES]
y_test = test[TARGETS]

print(y_test)

             LAT       LON
1694071  34.0256 -118.3089
1694072  33.7288 -118.2924
1694073  33.9572 -118.2782
1694074  34.3206 -118.4506
1694075  34.0782 -118.3604
...          ...       ...
2117584  34.0962 -118.3490
2117585  34.0871 -118.3732
2117586  34.0637 -118.2870
2117587  34.2266 -118.5085
2117588  34.0420 -118.4531

[423518 rows x 2 columns]


In [134]:
scalar= StandardScaler()
scalar.fit(X_train)
X_train=scalar.transform(X_train)
X_train

array([[-1.25500939,  1.00368457, -1.35761882, -1.31677016, -1.54269116],
       [ 0.68571532,  1.50696399,  0.43968774,  0.72060143, -1.54269116],
       [ 0.3622612 , -1.51271254,  0.43968774,  0.42954834, -1.54269116],
       ...,
       [ 0.34323449, -1.00943312,  0.43968774,  0.42954834,  1.83376473],
       [-0.25610697, -1.00943312, -0.45896554, -0.15255783,  1.83376473],
       [ 1.33262356, -1.51271254,  1.33834102,  1.3027076 ,  1.83376473]])

In [135]:
scalar= StandardScaler()
scalar.fit(X_test)
X_test=scalar.transform(X_test)
X_test

array([[-1.28133582e-01, -1.50522751e+00, -4.55626415e-01,
        -1.49489750e-01, -1.02709545e+00],
       [ 2.34938079e-01, -3.90850053e-04,  4.45027435e-01,
         1.42465205e-01, -1.02709545e+00],
       [ 1.63945214e+00, -3.90850053e-04,  1.34568129e+00,
         1.60223998e+00, -1.02709545e+00],
       ...,
       [-1.68551992e+00,  1.50444581e+00, -1.35628027e+00,
        -1.60926453e+00,  9.73619347e-01],
       [ 1.01840851e+00, -5.02003071e-01,  1.34568129e+00,
         1.01833007e+00,  9.73619347e-01],
       [-1.43710247e+00,  5.01221370e-01, -1.35628027e+00,
        -1.31730957e+00,  9.73619347e-01]])

In [136]:
reg = xgb.XGBRegressor()
reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        verbose=10)

[0]	validation_0-rmse:61.13595	validation_1-rmse:61.14538
[10]	validation_0-rmse:2.52579	validation_1-rmse:1.86703
[20]	validation_0-rmse:1.84396	validation_1-rmse:0.67205
[30]	validation_0-rmse:1.84317	validation_1-rmse:0.66853
[40]	validation_0-rmse:1.84306	validation_1-rmse:0.67070
[50]	validation_0-rmse:1.84300	validation_1-rmse:0.67081
[60]	validation_0-rmse:1.84295	validation_1-rmse:0.67133
[70]	validation_0-rmse:1.84291	validation_1-rmse:0.67143
[80]	validation_0-rmse:1.84288	validation_1-rmse:0.67146
[90]	validation_0-rmse:1.84285	validation_1-rmse:0.67166
[99]	validation_0-rmse:1.84283	validation_1-rmse:0.67201


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [137]:
coords=reg.predict(X_test)

In [138]:
coords

array([[  34.0595  , -118.28375 ],
       [  34.067482, -118.26673 ],
       [  34.086723, -118.37634 ],
       ...,
       [  34.07832 , -118.36798 ],
       [  34.079865, -118.36307 ],
       [  34.088535, -118.36673 ]], dtype=float32)

In [139]:
y_test

,LAT,LON
1694071,34.0256,-118.3089
1694072,33.7288,-118.2924
1694073,33.9572,-118.2782
1694074,34.3206,-118.4506
1694075,34.0782,-118.3604
...,...,...
2117584,34.0962,-118.3490
2117585,34.0871,-118.3732
2117586,34.0637,-118.2870
2117587,34.2266,-118.5085


In [140]:
p_df = pd.DataFrame(coords, columns=["LAT", "LON"])
len(p_df['LON'].unique())

689

In [141]:
# df = pd.DataFrame(y_test)
# df.to_csv('Actual Values overfit.csv')

# df = pd.DataFrame(coords)
# df.to_csv('Predicted Values overfit.csv')